In [23]:
# Credits: https://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/
# adapted from: - https://github.com/shshnk158/Amazon-Fine-Food-Reviews-Prediciton/blob/master/LSTM_Amazon_Food_Reviews.ipynb
#               - https://towardsdatascience.com/scikit-learn-for-text-analysis-of-amazon-fine-food-reviews-ea3b232c2c1b

# LSTM for sequence classification in the amazon fine-food reviews dataset
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

load data

In [24]:
X_train = pd.read_csv('../data/X_train.csv')
X_train = X_train['Text']
X_test = pd.read_csv('../data/X_test.csv')
X_test = X_test['Text']
y_train = pd.read_csv('../data/y_train.csv')
y_train = y_train['Helpful']
y_test = pd.read_csv('../data/y_test.csv')
y_test = y_test['Helpful']

tokenize

In [25]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)

In [26]:
X_test = tokenizer.texts_to_sequences(X_test)

In [27]:
print(X_train[45])
print(type(X_train[45]))
print(len(X_train[45]))

[3, 78, 3522, 4060, 66, 127, 1583, 214, 45, 223, 270, 10, 45, 4, 1182, 664, 3939, 1377, 135, 359, 41, 690, 296, 44, 127, 24, 649, 5, 3, 10, 1969, 3522, 316, 37, 1956, 554, 2, 73, 4061, 794, 8, 242, 62, 369]
<class 'list'>
44


In [28]:
# truncate and/or pad input sequences
max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

print(X_train.shape)
print(X_train[45])

(23885, 500)
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    

In [29]:
# create the model
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(5000, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy',
                                                                     tf.keras.metrics.SensitivityAtSpecificity(0.5),
                                                                     tf.keras.metrics.FalsePositives(),
                                                                     tf.keras.metrics.TrueNegatives()])
print(model.summary())
#Refer: https://datascience.stackexchange.com/questions/10615/number-of-parameters-in-an-lstm-model

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
lstm_6 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dropout_6 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None


In [30]:
model.fit(X_train, y_train, epochs=5, batch_size=64)

Epoch 1/5
23885/23885 [==============================] - 177s 7ms/step - loss: 0.3384 - accuracy: 0.8655 - sensitivity_at_specificity_4: 0.8180 - false_positives_2: 1476.8984 - true_negatives_2: 537.7380

In [31]:
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 89.12%


In [38]:
print(scores)

[0.3109912733147115, 0.8912333846092224, 0.9871014952659607, 7023.71484375, 13557.1923828125]


Two lstm layers

In [32]:
# embedding_vecor_length = 32
# model1 = Sequential()
# model1.add(Embedding(5000, embedding_vecor_length, input_length=max_review_length))
# model1.add(LSTM(100, return_sequences=True))
# model1.add(LSTM(100))
# model1.add(Dense(1, activation='sigmoid'))
# model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# print(model1.summary())

In [33]:
# model1.fit(X_train, y_train, epochs=5, batch_size=64)

In [34]:
# Final evaluation of the model
# scores = model1.evaluate(X_test, y_test, verbose=0)
# print("Accuracy: %.2f%%" % (scores[1]*100))

In [35]:
from sklearn.metrics import confusion_matrix

In [36]:
# confusion_matrix(y_test,model1.predict(X_test),normalize='true')


In [37]:
model.save('../models/LSTM_helpful_single')
# model1.save('../models/LSTM_helpful_2_layers')